## Setup

In [1]:
import boto3

session = boto3.Session(profile_name='bach-dev', region_name='us-east-1')
boto3_bedrock = session.client(service_name='bedrock-runtime')

In [2]:
from io import StringIO
import sys
import textwrap

def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))

## Configure langchain

In [3]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':200})
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

## Data Preparation

In [5]:
import numpy as np
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/imis_guide.pdf")
pages = loader.load()

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter( 
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(pages)

print(f"Split {len(pages)} pages into {len(docs)} chunks.")

Split 70 pages into 136 chunks.


In [6]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(pages)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(pages)} pages loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} chunks more than the original {len(pages)}.')
print(f'Average length among {len(docs)} chunks (after split) is {avg_char_count_post} characters.')

Average length among 70 pages loaded is 1451 characters.
After the split we have 136 chunks more than the original 70.
Average length among 136 chunks (after split) is 762 characters.


In [7]:
try:
    sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
    print("Sample embedding of a document chunk: ", sample_embedding)
    print("Size of the embedding: ", sample_embedding.shape)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

Sample embedding of a document chunk:  [ 0.10449219 -0.1484375  -0.31835938 ...  0.18554688 -0.38867188
 -0.484375  ]
Size of the embedding:  (1536,)


## Save to Chroma

In [9]:
from langchain_community.vectorstores import Chroma

CHROMA_PATH = "chroma"

db = Chroma.from_documents(docs, bedrock_embeddings, collection_metadata={"hnsw:space": "cosine"})
db.persist()

print(f"Saved {len(docs)} chunks to {CHROMA_PATH}.")

Saved 136 chunks to chroma.


## Question Answering

In [14]:
query = """What is the difference between email and eBill?"""
results = db.similarity_search_with_relevance_scores(query=query)

print_ww(results)

[(Document(page_content='15 | P a g e  \n  \n‘eBill’ is different from ‘Email.’ If you are making updates to eBill or an email address, ensure that you’re asking the \nMember if both should be updated.   \n                           \n \nCards & Bills  \nYou may order ad hoc C ards and Bills for members if required. You may either order Cards for the entire HH by \nselecting Household Card , or for an individual (s) by selecting Individual Card.  \n                                              \n \n \n       Household Card E xpiry  \n \nCards have a 3 year expiry and we print this replacement  date on the card.  If you order cards for any reason the system \nwill automatically check to see if the HH card expiry date is within 6 months from today.  If it is, it will not only order t he \ncard(s) you are requesting, it will advance the card expiry dat e by 3 years and order new cards for the entire HH.  \n \nIf this HH has a donor and requested  ‘Cards to:  Donor’, all 3 year replacement

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)
#print(prompt)


model = BedrockChat(model_id="anthropic.claude-v2", model_kwargs={"temperature": 0.1}, client=boto3_bedrock)
response_text = model.predict(prompt)

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print_ww(formatted_response)


c:\Users\eric.bach.CORP\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_community\llms\bedrock.py:57: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: 

Human: 
Answer the question based only on the following context:

15 | P a g e  
  
‘eBill’ is different from ‘Email.’ If you are making updates to eBill or an email address, ensure that you’re asking the 
Member if both should be updated.   
                           
 
Cards & Bills  
You may order ad hoc C ards and Bills for members if required. You may either order Cards for the entire HH by 
selecting Household Card , or for an individual (s) by selecting Individual Card.  
                                              
 
 
       Household Card E xpiry  
 
Cards have a 3 year expiry and we print this replacement  date on the card.  If you order cards for any reason the system 
will automatically check to see if the HH card expiry date is within 6 months from t

Response:  Based on the context provided, the difference between email and eBill is:

- Email refers to a regular email address for communication.

- eBill refers specifically to the electronic billing option that replaces the 1st paper bill for
AMA membership renewal. It allows members to go paperless and receive their first renewal notice
electronically instead of by mail.

So in summary, email is a general electronic communication method, while eBill is a specific
paperless billing option for AMA membership renewal notices.
Sources: ['data/imis_guide.pdf', 'data/imis_guide.pdf', 'data/imis_guide.pdf',
'data/imis_guide.pdf']


### Quick way

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
prompt_template = """

Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [13]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
answer = qa({"query": query})
print_ww(answer)

c:\Users\eric.bach.CORP\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'When are comments required?', 'result': ' Based on the context provided, comments are
required in the following situations:\n\n- Moves\n- Swapping Primary & Associate members but not
cancelling the former Primary \n- ATP move without changing the address or adding a wrong address
flag\n- Manual Transfer In (not using the Transfer In Wizard)\n\n- Roadside\n- Pre-existing
condition for new membership or upgrading \n\n- Payments\n- Leaving a household in Partial
Paid/Collect status (taking a partial payment only)\n- Leaving a household in Prospect or Unpaid
status (not taking payment)\n- Mailing a postdated cheque to PCC to hold for future processing
date\n\n- Creation of a 2nd iMIS ID intentionally \n- Transfer In that was previously an AMA
Member\n- Converting a Child to an Associate before age 16\n- ASSUME where the new person has a
suspended or cancelled membership', 'source_documents': [Document(page_content='18 | P a g e  \n  \n
\n      Comments  \n \nComments are used to

Let's ask a different question:

In [14]:
query_2 = "Can a member have multiple membership numbers?"
answer_2 = answer = qa({"query": query_2})
print_ww(answer_2)

{'query': 'Can a member have multiple membership numbers?', 'result': ' Based on the context
provided, the answer is yes, a member can have multiple membership numbers. The context mentions
that "A member may have multiple membership numbers but should always retain the same iMIS ID." It
also mentions reinstating a cancelled membership, which allows the member to keep their original
membership number unless they are moving to a new household. So in summary, a member can have
multiple membership numbers over time, but their iMIS ID remains constant.', 'source_documents':
[Document(page_content='7 | P a g e  \n  \n \n \n \n \n \n \n \n \n \n \n \n  \n \n \n  \n \n \nA
member may have multiple membership numbers  \nbut should always retain the same iMIS ID.  \n \n
\nGenerally if someone does not have a Membership # we display the iMIS ID instead.   \nThis applies
to Child  & Non-Member  Records.  \n \nQ: What if the Member is cancelled ?  Do we still display
their old Membership # or thei

### Customisable option
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the helpf of [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) which can be specific to the model.

Note: In this example we are using Anthropic Claude as the LLM under Amazon Bedrock. This particular model [performs best](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting) if the inputs are provided under `Human:` and the model is requested to generate an output after `Assistant:`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.

In [30]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """

Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
query = "What is the difference between ebill and email?"
result = qa({"query": query})
print(result['result'])
result['source_documents']

 Based on the context provided, the key difference between eBill and email is:

- eBill refers specifically to the option for AMA members to receive their annual renewal bill electronically instead of via physical mail. Choosing eBill replaces the paper bill that is normally sent out about 1 month before their membership expiry date.

- Email refers generically to a member's email address on file. The context indicates you should verify whether a member wants updates made to both their eBill preference and their email address on file if changes are requested.

So in summary, eBill is a billing preference for how to receive the annual renewal notice, while email is just the member's general email contact information.


[Document(page_content='15 | P a g e  \n  \n‘eBill’ is different from ‘Email.’ If you are making updates to eBill or an email address, ensure that you’re asking the \nMember if both should be updated.   \n                           \n \nCards & Bills  \nYou may order ad hoc C ards and Bills for members if required. You may either order Cards for the entire HH by \nselecting Household Card , or for an individual (s) by selecting Individual Card.  \n                                              \n \n \n       Household Card E xpiry  \n \nCards have a 3 year expiry and we print this replacement  date on the card.  If you order cards for any reason the system \nwill automatically check to see if the HH card expiry date is within 6 months from today.  If it is, it will not only order t he \ncard(s) you are requesting, it will advance the card expiry dat e by 3 years and order new cards for the entire HH.  \n \nIf this HH has a donor and requested  ‘Cards to:  Donor’, all 3 year replacement 